# Python Async BTree 'Sparemints

Messing with [async-btree](https://geronimo-iia.github.io/async-btree/) to see if we like it.

There is definite goodness in this library, but it's a bit stale and has few examples.

In [23]:
import async_btree as bt
import curio
import random

## Tutorial Code
Make sure it works.  Not always safe to assume, this was an example. They missed the `self` in the class below.

In [14]:
def approach_object(name: str):
    print(f"approach_object: {name}")
    return True
    
def check_battery():
    print("battery ok")
    return True

async def say_hello(name: str):
    print(f"Hello: {name}")
    return True
    
"""
approach_house_object_action = bt.action(target=approach_object, name="house")

check_battery_action = bt.action(target=check_battery)

say_hello_john = bt.action(target=say_hello, name="John")
"""


class GripperInterface:

    def __init__(self):
        self._open = False


    def open(self):
        print("GripperInterface Open")
        self._open = True
        return True

    def close(self):
        print("GripperInterface Close")
        self._open = False
        return True

gripper = GripperInterface()

b_tree = bt.sequence(children= [
    bt.action(target=say_hello, name="John"),
    bt.action(target=check_battery),
    bt.action(target=gripper.open),
    bt.action(target=approach_object, name="house"),
    bt.action(target=gripper.close)
])

curio.run(b_tree)

Hello: John
battery ok
GripperInterface Open
approach_object: house
GripperInterface Close


[True, True, True, True, True]

In [6]:
b_tree = bt.sequence(children= [
    bt.always_success(child=bt.action(target=say_hello, name="John")),
    bt.always_success(child=bt.action(target=check_battery)),
    bt.always_success(child=bt.action(target=gripper.open)),
    bt.always_success(child=bt.action(target=approach_object, name="house")),
    bt.always_success(child=bt.action(target=gripper.close))
])

curio.run(b_tree)

Hello: John
battery ok
GripperInterface Open
approach_object: house
GripperInterface Close


[True, True, True, True, True]

## Something Else

This is where the documentation kinda falls apart.

In [41]:
def max_damage(name: str):
    x = random.randint(1,4)
    if (x == 4):
        print("{n}: I rolled a 4!".format(n=name))
        return bt.SUCCESS
    else:
        print("{n}: Hold on, hit the pizza box".format(n=name))
        return bt.FAILURE

In [42]:
b_tree = bt.sequence(
    children = [
        bt.retry_until_success(bt.action(target=max_damage, name="Borfrim"))
    ]
)
curio.run(b_tree)

Borfrim: Hold on, hit the pizza box
result : False
Borfrim: Hold on, hit the pizza box
result : False
Borfrim: Hold on, hit the pizza box
result : False
Borfrim: Hold on, hit the pizza box
result : False
Borfrim: Hold on, hit the pizza box
result : False
Borfrim: I rolled a 4!
result : True


[True]